# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
! pip install cassandra-driver

You should consider upgrading via the 'C:\Users\Marsh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
# Import Python packages 
import datetime
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

C:\Users\Marsh\Desktop\2022-ApacheCassandra-ETL-DataModeling
C:\Users\Marsh\Desktop\2022-ApacheCassandra-ETL-DataModeling/event_data
['C:\\Users\\Marsh\\Desktop\\2022-ApacheCassandra-ETL-DataModeling/event_data\\2018-11-01-events.csv', 'C:\\Users\\Marsh\\Desktop\\2022-ApacheCassandra-ETL-DataModeling/event_data\\2018-11-02-events.csv', 'C:\\Users\\Marsh\\Desktop\\2022-ApacheCassandra-ETL-DataModeling/event_data\\2018-11-03-events.csv', 'C:\\Users\\Marsh\\Desktop\\2022-ApacheCassandra-ETL-DataModeling/event_data\\2018-11-04-events.csv', 'C:\\Users\\Marsh\\Desktop\\2022-ApacheCassandra-ETL-DataModeling/event_data\\2018-11-05-events.csv', 'C:\\Users\\Marsh\\Desktop\\2022-ApacheCassandra-ETL-DataModeling/event_data\\2018-11-06-events.csv', 'C:\\Users\\Marsh\\Desktop\\2022-ApacheCassandra-ETL-DataModeling/event_data\\2018-11-07-events.csv', 'C:\\Users\\Marsh\\Desktop\\2022-ApacheCassandra-ETL-DataModeling/event_data\\2018-11-08-events.csv', 'C:\\Users\\Marsh\\Desktop\\2022-ApacheCassandra-E

## Pandas Method - Processing the files to create the data file csv that will be used for Apache Casssandra tables 

In [4]:
# making a time stamp
now = datetime.datetime.now()
time_stamp = now.strftime("%c").replace(" ", "")
time_stamp = time_stamp.replace(":", "")

# for every filepath in the file path list 
combined_frame = pd.DataFrame()
for path in file_path_list:

# reading csv file 
    data = pd.read_csv(path,encoding = 'utf8')
    combined_frame = pd.concat([combined_frame, data], ignore_index=True)   
 # extracting each csv one by one and append it        
    #data.to_csv(f"{time_stamp}.csv", mode='a', index=False, encoding = 'utf8')

# saving the combined frame
combined_frame.to_csv(f"{time_stamp}.csv", mode='w', index=False, encoding = 'utf8')
combined_csv = f"{time_stamp}.csv"
print(combined_csv)

FriJan131330452023.csv


### Clean DATA , DROP unconstant Data, DROP Unused DATA

In [5]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format
data_frame = pd.read_csv(combined_csv)

#### Explore the Data & Check for Consistency

In [6]:
# How many rows and columns does df_apps have? What are the column names?
# What does the data look like?
# Look at a random sample of 5 different rows with
print(data_frame.shape)
print(data_frame.columns)
print(data_frame.sample())

(8056, 17)
Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userId'],
      dtype='object')
     artist       auth firstName gender  itemInSession lastName  length level  \
3567  Hevia  Logged In    Aleena      F              3    Kirby  256.47  paid   

                      location method      page         registration  \
3567  Waterloo-Cedar Falls, IA    PUT  NextSong 1,541,020,000,000.00   

      sessionId                 song  status                   ts  userId  
3567        637  Fandangu Los Llobos     200 1,542,320,000,000.00   44.00  


#### Dropping the columns that are not relevant to our queries  

In [7]:
# drop the cols that are not needed
data_frame.drop(["auth", "method", "page", "registration", "status", "ts"], axis=1, inplace=True)
print(data_frame.head())

    artist firstName gender  itemInSession lastName  length level  \
0      NaN    Walter      M              0     Frye     NaN  free   
1      NaN    Kaylee      F              0  Summers     NaN  free   
2  Des'ree    Kaylee      F              1  Summers  246.31  free   
3      NaN    Kaylee      F              2  Summers     NaN  free   
4  Mr Oizo    Kaylee      F              3  Summers  144.04  free   

                            location  sessionId          song  userId  
0  San Francisco-Oakland-Hayward, CA         38           NaN   39.00  
1        Phoenix-Mesa-Scottsdale, AZ        139           NaN    8.00  
2        Phoenix-Mesa-Scottsdale, AZ        139  You Gotta Be    8.00  
3        Phoenix-Mesa-Scottsdale, AZ        139           NaN    8.00  
4        Phoenix-Mesa-Scottsdale, AZ        139       Flat 55    8.00  


#### Cleaning the data here

In [8]:
# Remove nun & dublicate values
print(f'Missing values for data set?: {data_frame.isna().values.any()}')
print(f'Missing values for data set?: {data_frame.isna().values.sum()}')
data_frame.dropna(inplace=True)
print(f'Missing values for data set?: {data_frame.isna().values.any()}')
print(f'Missing values for data set?: {data_frame.isna().values.sum()}')
# Remove duplicates
print(f'duplicates values for data set?: {data_frame.duplicated().values.any()}')
print(f'duplicates values for data set?: {data_frame.duplicated().values.sum()}')
print(data_frame.duplicated)  


Missing values for data set?: True
Missing values for data set?: 5138
Missing values for data set?: False
Missing values for data set?: 0
duplicates values for data set?: False
duplicates values for data set?: 0
<bound method DataFrame.duplicated of                  artist firstName gender  itemInSession lastName  length  \
2               Des'ree    Kaylee      F              1  Summers  246.31   
4               Mr Oizo    Kaylee      F              3  Summers  144.04   
5            Tamba Trio    Kaylee      F              4  Summers  177.19   
6        The Mars Volta    Kaylee      F              5  Summers  380.42   
7     Infected Mushroom    Kaylee      F              6  Summers  440.27   
...                 ...       ...    ...            ...      ...     ...   
8050       Foo Fighters     Rylan      M             57   George  271.39   
8051         Timbiriche     Rylan      M             58   George  202.61   
8052   A Perfect Circle     Rylan      M             59   George  

#### check our data after  cleanning 

In [9]:
print(data_frame.shape)
print(data_frame.columns)
print(data_frame.sample())

(6820, 11)
Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')
        artist firstName gender  itemInSession lastName  length level  \
6547  Deadmau5     Tegan      F              6   Levine  429.95  paid   

                         location  sessionId    song  userId  
6547  Portland-South Portland, ME        933  Arguru   80.00  


#### Change Columns type to the desired format

In [10]:
print(data_frame.describe())
print(data_frame.info())

       itemInSession   length  sessionId   userId
count       6,820.00 6,820.00   6,820.00 6,820.00
mean           22.76   247.03     599.18    54.68
std            23.44   102.98     284.95    28.16
min             0.00    15.86       3.00     2.00
25%             4.00   197.32     374.00    29.00
50%            15.00   232.97     605.00    49.00
75%            35.00   274.12     834.00    80.00
max           126.00 2,594.87   1,114.00   101.00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 2 to 8055
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6820 non-null   object 
 1   firstName      6820 non-null   object 
 2   gender         6820 non-null   object 
 3   itemInSession  6820 non-null   int64  
 4   lastName       6820 non-null   object 
 5   length         6820 non-null   float64
 6   level          6820 non-null   object 
 7   location       6820 non-null   object 
 8 

In [11]:
data_frame["itemInSession"] = pd.to_numeric(data_frame["itemInSession"])
data_frame["length"] = pd.to_numeric(data_frame["length"])
data_frame["sessionId"] = pd.to_numeric(data_frame["sessionId"])
data_frame["userId"] = pd.to_numeric(data_frame["userId"],downcast='integer' )


#### See the new data types and if the data is ready

In [12]:
print(data_frame.describe())
print(data_frame.info())

       itemInSession   length  sessionId   userId
count       6,820.00 6,820.00   6,820.00 6,820.00
mean           22.76   247.03     599.18    54.68
std            23.44   102.98     284.95    28.16
min             0.00    15.86       3.00     2.00
25%             4.00   197.32     374.00    29.00
50%            15.00   232.97     605.00    49.00
75%            35.00   274.12     834.00    80.00
max           126.00 2,594.87   1,114.00   101.00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 2 to 8055
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6820 non-null   object 
 1   firstName      6820 non-null   object 
 2   gender         6820 non-null   object 
 3   itemInSession  6820 non-null   int64  
 4   lastName       6820 non-null   object 
 5   length         6820 non-null   float64
 6   level          6820 non-null   object 
 7   location       6820 non-null   object 
 8 

In [13]:
data_frame.to_csv(f"event_datafile_new_{time_stamp}.csv", mode='w', index=False, encoding = 'utf8')

## Method 2 - Processing the files to create the data file csv that will be used for Apache Casssandra tables 

In [ ]:
'''
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
'''

In [ ]:
'''
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
    
'''

# Part II. Model & Denormalize the DB for the Needed Queries , Optimizing for Fast, Responsive Reads

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [25]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    #cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    cluster = Cluster(['127.0.0.1'], port='1')
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()

except Exception as e:
    print(e)



#### Create Keyspace

In [37]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cass_etl 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [38]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("cass_etl")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [41]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_app_history"
query = query + """(artist TEXT, song TEXT,
                    length FLOAT, 
                    sessionId INT, itemInSession INT, 
                    PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e :
    print(e)
                    

In [47]:
for index, row in data_frame.iterrows():   
    query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    try:
        session.execute(query, (row["sessionId"], row["itemInSession"], row["artist"], row["song"], row["length"]))
    except Exception as e:
        print(e)
"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
"""        


#### Do a SELECT to verify that the data have been inserted into each table

In [55]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length, sessionId , itemInSession FROM music_app_history WHERE sessionId = 338 AND itemInSession = 4"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
df = pd.DataFrame(results._current_rows)
print(df)

      artist                             song  length  sessionid  \
0  Faithless  Music Matters (Mark Knight Dub)  495.31        338   

   iteminsession  
0              4  


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [62]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS user_app_history"       
query = query + """(artist TEXT, song TEXT, firstName TEXT, lastName TEXT , 
                    sessionId INT, itemInSession INT, userId INT, 
                    PRIMARY KEY ((userId, sessionId), itemInSession))"""
try:
    session.execute(query)
except Exception as e :
    print(e)

In [63]:
for index, row in data_frame.iterrows():   
    query = "INSERT INTO user_app_history (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    try:
        session.execute(query, (row["userId"], row["sessionId"], row["itemInSession"], row["artist"], row["song"], row["firstName"], row["lastName"]))
    except Exception as e:
        print(e)
        
"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_app_history (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
"""

In [65]:
query = "SELECT artist, song, firstName, lastName FROM user_app_history WHERE userId = 10 AND sessionId = 182"
try:
    results = session.execute(query)
except Exception as e :
    print(e)
df = pd.DataFrame(results._current_rows)
print(df)

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


In [66]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_info"       
query = query + """(song TEXT, firstName TEXT, lastName TEXT , 
                    userId INT, 
                    PRIMARY KEY (song,userId))"""
try:
    session.execute(query)
except Exception as e :
    print(e)

                    

In [67]:
for index, row in data_frame.iterrows():   
    query = "INSERT INTO user_info (userId, song, firstName, lastName)"
    query = query + "VALUES (%s, %s, %s, %s)"
    try:
        session.execute(query, (row["userId"], row["song"], row["firstName"], row["lastName"]))
    except Exception as e:
        print(e)
"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_users (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
"""

In [ ]:
query = "SELECT firstName, lastName FROM user_info WHERE  song = 'All Hands Against His Own'"
try:
    results = session.execute(query)
except Exception as e :
    print(e)
df = pd.DataFrame(results._current_rows)
print(df)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
query = "drop table music_app_history"
try:
    rows = session.execute(query)
except Exceptio

query = "drop table user_app_history"
try:
    rows = session.execute(query)
except Exceptio

query = "drop table user_info"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
#execute("UPDATE limit_data set value = %s where user_id = %s and limit_data.key = %s",(0,session['user_id'],'limit_download_result'),)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()